Descripción general del sistema de Speech Denoising

El sistema está diseñado para realizar reducción de ruido en audio mediante técnicas de deep learning. La entrada del modelo corresponde al espectro de magnitudes de la señal de audio con ruido, mientras que la salida objetivo corresponde al espectro de magnitudes de la señal limpia. La red neuronal toma como entrada el espectro de magnitudes con ruido y estima el espectro de magnitudes de la señal denoised.
El entrenamiento se realiza minimizando el error cuadrático medio (MSE) entre el espectro predicho y el espectro objetivo. La arquitectura de la red está compuesta por capas totalmente conectadas que procesan segmentos de la señal en el dominio frecuencia–tiempo.

Tras la inferencia, la señal denoised se reconstruye en el dominio del tiempo combinando el espectro de magnitudes predicho con la información de fase original de la señal con ruido. Esta reconstrucción se lleva a cabo mediante la transformada rápida de Fourier inversa de ventana corta (ISTFT). El resultado es una forma de onda con el ruido significativamente reducido, y que se aproxima a la señal limpia de referencia.

In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import Utils.model_utils

In [ ]:
import torch

window_size = 256
model_params= {"window_length": window_size,
"overlap": int(0.75 * window_size),
"fft_length": window_size,
"num_features": window_size // 2 + 1,
"num_segments": 8,
"batch_size": 128,
"epochs": 3,
"lr": 1e-5}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clean_folder = "../Audios/Dataset/dev-clean"

noisy_folder = "../Audios/Augmented Audios 3 SNR"

noisy_segments_list, clean_segments_list = Utils.model_utils.initialize(model_params, clean_folder, noisy_folder)

X, y, noisy_mean, noisy_std, clean_mean, clean_std = Utils.model_utils.batch_preprocessing(clean_segments_list, noisy_segments_list)

model = Utils.model_utils.DenoiseNet(model_params["num_features"], model_params["num_segments"]).to(device)

train_loader, val_loader = Utils.model_utils.dataset_split(X, y)


Epoch 1/3 - Train Loss: 0.319473 - Val Loss: 0.137561
Epoch 2/3 - Train Loss: 0.136529 - Val Loss: 0.093674
Epoch 3/3 - Train Loss: 0.106347 - Val Loss: 0.077738


In [ ]:
trained_model = Utils.model_utils.train_model(model, model_params["epochs"], model_params, train_loader, val_loader)

In [ ]:
model_data = {
    "model_state": model.state_dict(),
    "model_params": model_params,
    "normalization": {"noisy_mean": noisy_mean, "noisy_std": noisy_std, "clean_mean": clean_mean, "clean_std": clean_std},
    "other_info": "Modelo entrenado con dataset LibriSpeech"
}

torch.save(model_data, "denoise_checkpoint.pth")

print(f"model saved")
